# Laboratorium nr 8

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Skalowanie danych
Proszę pamiętać, aby zawsze estymować parametry skalowania (średnia i wariancja) na zbiorze treningowym, a nie na całym zbiorze danych. W przeciwym razie, zbierając statystyki z całego zbioru danych, "przenosimy" część informacji ze zbioru testowego/walidacyjnego do treningowego, co jest błędem metodologicznym.

### Zadanie nr 1 (1 punkt)
Załaduj dane _breast_cancer_, a następnie naucz model SVC na danych (tylko zmienne niezależne!) zestandaryzowanych i niezestandaryzowanych. Użyj w tym celu StandardScalera. Czy jest jakaś różnica w predykcji?

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

cancer = load_breast_cancer()

# get the data
X = cancer.data
y = cancer.target

# split it into training and test sets
X_train, X_test = train_test_split(X, random_state=5, test_size=.1)

## Stratified K fold
Wyobraź sobie, że masz torbę pełną kolorowych kulek: 90 czerwonych kulek i 10 niebieskich. Jeśli podzielisz kulki losowo na 5 grup, może się zdarzyć, że w niektórych grupach w ogóle nie będzie niebieskich kulek! Ale jeśli podzielisz kulki uważnie, tak aby w każdej grupie było około 90% czerwonych i 10% niebieskich kulek — tak jak w oryginalnej torbie — wtedy proporcje zostaną zachowane. Właśnie to robi *Stratified K-Fold*! Zachowuje oryginalny rozkład klas w każdym foldzie (podziale). To jest szczególnie ważne, gdy dane są niezbalansowane.

## Metryki
Do oceny jakości modelu używamy następujących metryk:
1. Accuracy mierzy procent poprawnych predykcji w stosunku do wszystkich predykcji.
$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN},
$$ gdzie:
- TP (True Positive) – poprawnie przewidziane pozytywne przypadki
- TN (True Negative) – poprawnie przewidziane negatywne przypadki
- FP (False Positive) – błędnie przewidziane pozytywne przypadki
- FN (False Negative) – błędnie przewidziane negatywne przypadki
2. Precision mierzy, jaki procent przewidywanych jako pozytywne przykładów faktycznie jest pozytywny.
$$
\text{Precision} = \frac{TP}{TP + FP}.
$$
3. Recall mierzy, jaki procent rzeczywistych pozytywnych przypadków został poprawnie wykryty przez model.
$$
\text{Recall} = \frac{TP}{TP + FN}.
$$

4. F1-score jest średnią harmoniczną precision i recall.
$$
\text{F1\ Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}.
$$

### Zadanie nr 2 (5 punktów)
Instrukcje:
1. Załaduj zbiór danych Iris.
2. Sztucznie zaburz balans klas:
    - Nie wyrzucaj danych z jednej klasy.
    - Z dwóch pozostałych klas zostaw tylko 30% próbek.

Użyj dwóch metod:
- KFold (zwykły podział).
- StratifiedKFold (podział ze stratfikacją).

Następnie dla każdej metody:
1. Przeprowadź walidację krzyżową (5-fold) z użyciem regresji logistycznej.
2. Wydrukuj rozkład klas w każdym foldzie.
3. Wydrukuj dla każdego folda:
    - Dokładność klasyfikacji (accuracy).
    - Precision.
    - Recall.
    - F1 score.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import Counter

## Krzywa ROC i AUC

**ROC (Receiver Operating Characteristic curve)** to wykres pokazujący zależność pomiędzy:

- **True Positive Rate (TPR)** — czyli czułość (recall)
- **False Positive Rate (FPR)** — odsetek fałszywych pozytywów

dla różnych progów decyzyjnych.

---

### Wzory:

**True Positive Rate (TPR):**

$$
\text{TPR} = \frac{TP}{TP + FN}
$$

**False Positive Rate (FPR):**

$$
\text{FPR} = \frac{FP}{FP + TN}
$$

Gdzie:
- **TP** (True Positive) — prawidłowo przewidziane pozytywne przypadki
- **FP** (False Positive) — błędnie przewidziane pozytywne przypadki
- **FN** (False Negative) — błędnie przewidziane negatywne przypadki
- **TN** (True Negative) — prawidłowo przewidziane negatywne przypadki

---

## Co to jest AUC?

**AUC (Area Under the Curve)** to **pole pod krzywą ROC**.

- AUC = 1.0 oznacza idealnego klasyfikatora.
- AUC = 0.5 oznacza losowe zgadywanie.
- Im wyższe AUC, tym lepsza jakość klasyfikatora.

---

## Dlaczego używamy ROC i AUC?

- Są **niezależne od konkretnego progu** decyzyjnego.
- Bardzo przydatne przy **niezbalansowanych danych**, gdy accuracy może wprowadzać w błąd.
- Pozwalają ocenić, jak dobrze klasyfikator odróżnia pozytywne i negatywne przykłady.


### Zadanie 3 (4 punkty)
1. Załaduj zbiór danych Iris.
2. Zrób klasyfikację tylko dla dwóch klas (np. klasa 0 vs. reszta).
3. Podziel dane na zbiór treningowy i testowy.
4. Zbuduj Pipeline, składający się z obiektów:
    - StandardScaler (skalowanie)
    - LogisticRegression (regresja logistyczna jako klasyfikator)
    - Przy pomocy GridSearchCV znajdź najlepsze parametry dla regresji logistycznej (C, penalty).
5. Oceń wyniki na zbiorze testowym.
6. Narysuj krzywą ROC i podaj AUC (Area Under Curve).

In [ ]:
# Do budowy pipeline'u. Żeby zobaczyć, jak go używać, sprawdź dokumentację.
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score